# C2Q Method 2

v11

In [3]:
import numpy as np
from thesis.quantum import random_data
from IPython.display import Markdown as md
    
def partial_measurement(psi_in, bits_to_remove):
    psi_out = np.zeros_like(psi_in)  # Output vector with same length as psi_in

    # Create bitmask from bits_to_remove
    bitmask = sum(2**x for x in bits_to_remove)
    bitmask = ~bitmask  # Negative mask of bitmask

    # Reduce psi_in while preserving indices
    for i, x in enumerate(psi_in):
        i_out = i & bitmask  # index in psi_out to insert value
        psi_out[i_out] += (
            np.abs(x) ** 2
        )  # Add the square of the value in psi (probability)
    psi_out = np.sqrt(psi_out)  # Take sqrt to return to a "statevector"

    return psi_out

x_in = random_data(2, is_complex=True, seed=42069)
# x_in = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0]
# x_in = partial_measurement(x_in, [1])

print(x_in)

[0.03050837-0.35203536j 0.4194086 +0.53065957j 0.87769215-0.50648638j
 0.71350281+0.8650064j ]


In [4]:
# Format input array properly
x_in = np.array(x_in)

# Calculate number of necessary qubits and states
num_qubits = int(np.ceil(np.log2(len(x_in))))
num_states = np.power(2, num_qubits, dtype=int)

# Pad input array if necessary
if num_states - len(x_in) != 0: 
    print(f'Input length is not power of 2. Padding...')
    x_in = np.pad(x_in, (0, num_states-len(x_in)), 'constant', constant_values=0)
    print(f'New input: {x_in}')

# Normalize input array if necessary
if np.linalg.norm(x_in) != 1:
    print(f'Input is not normalized. Normalizing...')
    x_in = x_in / np.linalg.norm(x_in)
    print(f'New input: {x_in}')

Input is not normalized. Normalizing...
New input: [0.01801933-0.20792465j 0.2477177 +0.31342649j 0.51839633-0.29914894j
 0.42142024+0.51090367j]


In [5]:
### ============================ ###
###  Calculate input parameters  ###
### ============================ ###

# Generates P matrix as in Eq. 17
# Returns tuple of alpha and beta matrices from P, see Eqs. 18 and 19
def get_ab(x_in):
    x_new = np.reshape(x_in, (int(len(x_in)/2), 2))
    i_max = int(len(x_new))
    j_max = int(np.ceil(np.log2(len(x_in))))

    P = np.zeros((i_max, j_max), dtype=complex)
    alpha = np.zeros((i_max, j_max), dtype=complex)
    beta = np.zeros((i_max, j_max), dtype=complex)
    for j in range(j_max):
        
        # TODO: this portion may have optimization potential
        for (i, x) in enumerate(x_new):
            if j == 0:
                p = np.power(np.linalg.norm(x), 2)

                if p == 0:
                    a = 1
                    b = 0
                else:
                    a = x[0] / np.linalg.norm(x)
                    b = x[1] / np.linalg.norm(x)
            elif i >= i_max / (2**j):
                p = 0 
                a = 1
                b = 0
            else:
                p = P[2*i,j-1] + P[2*i+1,j-1]

                if p == 0:
                    a = 1
                    b = 0
                else:  
                    a = np.sqrt(P[2*i,j-1] / p)
                    b = np.sqrt(P[2*i+1,j-1] / p)

            # This is purely done for readability    
            P[i,j] = p
            alpha[i,j] = a
            beta[i,j] = b

    return (alpha, beta)

# Returns values of theta, phi, r, and t according to Eq. 20
def get_params(alpha, beta):
    print(np.abs(alpha))
    print(np.abs(beta))
    
    alpha_mag = np.abs(alpha)
    alpha_phase = np.angle(alpha)
    beta_mag = np.abs(beta)
    beta_phase = np.angle(beta)

    with np.errstate(divide='ignore'):
        theta = 2*np.arctan(beta_mag/alpha_mag)
    phi = beta_phase - alpha_phase
    r = np.sqrt(alpha_mag**2 + beta_mag**2)
    t = beta_phase + alpha_phase

    return theta, phi, r, t

# Returns tuple of theta, phi, r, and t tensors given input data
def input_data(x_in):
    return get_params(*get_ab(x_in))

theta_array, phi_array, r_array, t_array = input_data(x_in)

[[0.46303545 0.45073005]
 [0.67048895 1.        ]]
[[0.88633976 0.8926603 ]
 [0.74191952 0.        ]]


In [6]:
# Basic gates

def Ry(theta):
    return np.matrix(f'{np.cos(theta/2)}, {-np.sin(theta/2)}; {np.sin(theta/2)}, {np.cos(theta/2)}')

def Rz(phi):
    return np.matrix(f'{np.exp(-1j*phi/2)}, {0}; {0}, {np.exp(1j*phi/2)}')

def Uij(theta, phi, r, t):
    #return r * np.exp(1j*t/2) * (Rz(phi) @ Ry(theta))
    return (Rz(phi) @ Ry(theta) @ Rz(-t)) * r
    # return (Rz(phi-t) @ Ry(theta)) * r

In [7]:
# Contruct each Uj in pyramidal structure
Uj_array = []

for j in range(num_qubits):
    n_j = num_qubits - 1 - j
    i_max = 2**(n_j)

    theta_j = theta_array[0:i_max,j]
    phi_j = phi_array[0:i_max,j]
    r_j = r_array[0:i_max,j]
    t_j = t_array[0:i_max,j]
    
    Uj = np.asmatrix(np.zeros((2*i_max, 2*i_max)), dtype=complex)
    for i, (theta, phi, r, t) in enumerate(zip(theta_j, phi_j, r_j, t_j)):
        Uj[2*i:2*i+2, 2*i:2*i+2] = Uij(theta, phi, r, t)

    Uj_array.append(Uj)
    # print(Uj)

In [8]:
# 1 qubit |0> state
zero_state = np.matrix('1;0')

# Create initial quantum state
psi_0 = zero_state
for _ in range(num_qubits-1):
    psi_0 = np.kron(zero_state, psi_0)

In [9]:
# Find matrix for entire circuit
U_block = np.eye(2**num_qubits)
for (n_j, U_j) in enumerate(reversed(Uj_array)):
    U_j_kron = np.kron(U_j, np.eye(2**(num_qubits - 1 - n_j)))
    U_block = U_j_kron @ U_block

# Encode state
psi = U_block @ psi_0

In [10]:
# Helper function for printing matrix in LATEX format
def matrix(M: np.matrix) -> str:
    output = '\\begin{bmatrix}'

    n_cols = 1
    if len(M.shape) == 2:
        _, n_cols = M.shape

    for i, m in enumerate(np.nditer(M)):
        output += f'{m:.03f}'

        if ((i+1) % n_cols) == 0:
            output += '\\\\'
        else:
            output += '&'

    output += '\\end{bmatrix}'

    return output

# Test if matrices are unitary
def is_unitary(M: np.matrix) -> bool:
    M = np.asmatrix(M)

    if len(M.shape) != 2:
        return False;

    n_rows, n_cols = M.shape

    if n_rows != n_cols:
        return False

    M_squared = M @ M.getH()
    M_squared = M_squared.round(3)

    # print(M_squared)

    return np.array_equal(M_squared, np.eye(n_rows))    

In [11]:
# Print important values in LATEX

md_str  = f'Input data: ${matrix(x_in)}$, '
md_str += f'Number of qubits: {num_qubits}\n'

md_str += '\n'

md_str += f'Matrix of parameters: \n\n'
md_str += f'$\\theta = {matrix(theta_array)}$, '
md_str += f'$\phi = {matrix(phi_array)}$, '
md_str += f'$t = {matrix(t_array)}$, '

# I don't know how to fix r
md_str += f'$r = {matrix(r_array)}$\n'

md_str += '\n'

md_str += f'Each $U_j$ matrix defined in C2Q Method 2: \n\n'
for j, Uj in enumerate(Uj_array):
    md_str += f'$U_{j} = {matrix(Uj)}$\n'
    md_str += f'(Unitary: {is_unitary(Uj)})\n\n'

md_str += '\n'

md_str += f'The matrix for the circuit overall: '
md_str += f'$\left( U^{{C2Q-2}}\lvert 0 \\rangle^{{\otimes n}} = \\vert \Psi \\rangle \\right)$ \n\n'
md_str += f'$U^{{C2Q-2}} = {matrix(U_block)}$\n'
md_str += f'(Unitary: {is_unitary(U_block)})\n\n'

md_str += '\n'

md_str += f'Final encoded state: \n\n'
md_str += f'$\lvert \Psi \\rangle = {matrix(psi)}$\n'

md_str += '\n'

md_str += f'Is final state correct?: {np.allclose(x_in, np.transpose(psi))}\n'

md(md_str)

Input data: $\begin{bmatrix}0.018-0.208j\\0.248+0.313j\\0.518-0.299j\\0.421+0.511j\\\end{bmatrix}$, Number of qubits: 2

Matrix of parameters: 

$\theta = \begin{bmatrix}2.179&2.206\\1.672&0.000\\\end{bmatrix}$, $\phi = \begin{bmatrix}2.386&0.000\\1.404&0.000\\\end{bmatrix}$, $t = \begin{bmatrix}-0.582&0.000\\0.358&0.000\\\end{bmatrix}$, $r = \begin{bmatrix}1.000&1.000\\1.000&1.000\\\end{bmatrix}$

Each $U_j$ matrix defined in C2Q Method 2: 

$U_0 = \begin{bmatrix}0.040-0.461j&-0.550+0.695j&0.000+0.000j&0.000+0.000j\\0.550+0.695j&0.040+0.461j&0.000+0.000j&0.000+0.000j\\0.000+0.000j&0.000+0.000j&0.581-0.335j&-0.472+0.572j\\0.000+0.000j&0.000+0.000j&0.472+0.572j&0.581+0.335j\\\end{bmatrix}$
(Unitary: True)

$U_1 = \begin{bmatrix}0.451+0.000j&-0.893+0.000j\\0.893+0.000j&0.451+0.000j\\\end{bmatrix}$
(Unitary: True)


The matrix for the circuit overall: $\left( U^{C2Q-2}\lvert 0 \rangle^{\otimes n} = \vert \Psi \rangle \right)$ 

$U^{C2Q-2} = \begin{bmatrix}0.018-0.208j&-0.248+0.313j&-0.036+0.412j&0.491-0.621j\\0.248+0.313j&0.018+0.208j&-0.491-0.621j&-0.036-0.412j\\0.518-0.299j&-0.421+0.511j&0.262-0.151j&-0.213+0.258j\\0.421+0.511j&0.518+0.299j&0.213+0.258j&0.262+0.151j\\\end{bmatrix}$
(Unitary: True)


Final encoded state: 

$\lvert \Psi \rangle = \begin{bmatrix}0.018-0.208j\\0.248+0.313j\\0.518-0.299j\\0.421+0.511j\\\end{bmatrix}$

Is final state correct?: True
